In [ ]:
import pandas as pd
import numpy as np

# more libraries probably... 
import tensorflow

import textblob
import json
from nltk.tokenize import word_tokenize
import nltk


In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.3 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


# Evaluation

In [ ]:

# Evaluation metric

import sys
import os
import os.path
from scipy.stats import sem
import numpy as np
from ast import literal_eval

def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)


def evaluate(pred, gold):
    """
    Based on https://github.com/felipebravom/EmoInt/blob/master/codalab/scoring_program/evaluation.py
    :param pred: file with predictions
    :param gold: file with ground truth
    :return:
    """
    # # read the predictions
    # pred_lines = pred.readlines()
    # # read the ground truth
    # gold_lines = gold.readlines()

    pred_lines = pred
    gold_lines = gold

    # only when the same number of lines exists
    if (len(pred_lines) == len(gold_lines)):
        data_dic = {}
        for n, line in enumerate(gold_lines):
            parts = line.split('\t')
            if len(parts) == 2:
                data_dic[int(parts[0])] = [literal_eval(parts[1])]
            else:
                raise ValueError('Format problem for gold line %d.', n)

        for n, line in enumerate(pred_lines):
            parts = line.split('\t')
            if len(parts) == 2:
                if int(parts[0]) in data_dic:
                    try:
                        data_dic[int(parts[0])].append(literal_eval(parts[1]))
                    except ValueError:
                        # Invalid predictions are replaced by a default value
                        data_dic[int(parts[0])].append([])
                else:
                    raise ValueError('Invalid text id for pred line %d.', n)
            else:
                raise ValueError('Format problem for pred line %d.', n)

        # lists storing gold and prediction scores
        scores = []
        for id in data_dic:
            if len(data_dic[id]) == 2:
                gold_spans = data_dic[id][0]
                pred_spans = data_dic[id][1]
                scores.append(f1(pred_spans, gold_spans))
            else:
                sys.exit('Repeated id in test data.')

        return (np.mean(scores), sem(scores))


In [ ]:
#max comment length
max_len = 128 

# max embedding dimension(for one word)
embedding_dim = 25 
# Max feature
max_feature = 10000


In [ ]:



# Read data
import pandas as pd
from ast import literal_eval

data = pd.read_csv("ML_Final_Proj1/data1/tsd_train.csv")
dev = pd.read_csv("ML_Final_Proj1/data1/tsd_trial.csv")
test = pd.read_csv("ML_Final_Proj1/data1/tsd_test.csv")

text_data = data['text'].values
spans = data['spans'].apply(literal_eval)
lbl = [1 if len(s) > 0 else 0 for s in spans]

text_data_test = test['text'].values
spans_test = test['spans'].apply(literal_eval)
test_id = test.index
lbl_test = [1 if len(s) > 0 else 0 for s in spans_test]

text_data_dev = dev['text'].values
spans_dev = dev['spans'].apply(literal_eval)
dev_id = dev.index
lbl_dev = [1 if len(s) > 0 else 0 for s in spans_dev]

In [ ]:

# Token level 

from nltk.tokenize import TweetTokenizer
import numpy as np
import spacy

tknzr2 = TweetTokenizer()

def custom_tokenizer(text_data):
    return tknzr2.tokenize(text_data)

def retrieve_word_from_span(lst_span, text):
    i = 0
    token = []
    a = 0

    word = []

    while (i < (len(lst_span) - 1)):
        if (lst_span[i] != (lst_span[i+1]-1)):
            token.append(lst_span[a:(i+1)])
            a = i + 1
        elif i == (len(lst_span) - 2):
            token.append(lst_span[a:i+2])

        i = i + 1

    for t in token:
        word.append(text[t[0]:(t[len(t)-1])+1])

    return word

def span_retrived(text_data, spans):
    token_labels = []

    for i in range(0, len(text_data)):
        token_labels.append(retrieve_word_from_span(spans[i], text_data[i]))
    
    return token_labels

def span_convert(text_data, spans):
    MAX_LEN = 0
    token_labels = []

    for i in range(0, len(text_data)):
        token_labels.append(retrieve_word_from_span(spans[i], text_data[i]))

    lst_seq = []
    for i in range(0, len(text_data)):
        # token = tknzr.tokenize(text_data[i])
        token = custom_tokenizer(text_data[i])
        if len(token) > MAX_LEN:
            MAX_LEN = len(token)
                   
        seq = np.zeros(len(token), dtype=int)
        for j in range(0, len(token)):
            for t in token_labels[i]:
                # if token[j] in tknzr.tokenize(t):
                if token[j] in custom_tokenizer(t):
                    seq[j] = 1
        lst_seq.append(seq)     

    return (token_labels, lst_seq)

In [ ]:
from copy import deepcopy

# convert data
data['token'], data['seq'] = span_convert(text_data, spans)
dev['token'], dev['seq'] = span_convert(text_data_dev, spans_dev)
test['token'], test['seq'] = span_convert(text_data_test, spans_test)

train = deepcopy(data)
data = pd.concat([data, dev])
     


# Embedding words


In [ ]:
# Read embedding
word_dict = []
embeddings_index = {}
f = open('ML_Final_Proj1/data1/glove.twitter.27B.25d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] 
    word_dict.append(word)
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('GloVe success')



GloVe success


In [ ]:
words = word_dict
num_words = len(words)

# Dictionary word:index pair
# word is key and its value is corresponding index
word_to_index = {w : i + 2 for i, w in enumerate(words)}
word_to_index["UNK"] = 1
word_to_index["PAD"] = 0

# Dictionary lable:index pair
idx2word = {i: w for w, i in word_to_index.items()}
     


In [ ]:
# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_to_index.items():
    if i > max_feature:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)
     

 # mapping for token cases
case2Idx = {'1': 1, '0': 0}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')  # identity matrix used 

char2Idx = {"PADDING": 0, "UNKNOWN": 1}
for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\%$`&=*+@^~|<>":
    char2Idx[c] = len(char2Idx)
     

In [ ]:
import keras.preprocessing.sequence

Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split

y = data['seq']
X = data['text']

y_test = test['seq']
X_test = test['text']
     

#train test
from sklearn.model_selection import train_test_split

X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size = 0.1)
     

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer


from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.initializers import Constant
from nltk.corpus import stopwords
import re
import numpy as np

tknzr2 = TweetTokenizer()

def custom_tokenizer(text_data):
    text_data = text_data.lower()
    return tknzr2.tokenize(text_data)

def preprocess(text):
    text = text.lower()

    word_list = nltk.word_tokenize(text)
    lemma = WordNetLemmatizer()

    for w in word_list:
        w = lemma.lemmatize(w)

    new_text = ""
    for w in word_list:
        new_text = new_text + " " + w

    return new_text

def encoding(X, y, isTest = True):
    sentences = []
    
    for t in X:
        sentences.append(custom_tokenizer(t))

    X = []
    for s in sentences:
        sent = []
        for w in s:
            try:
                w = w.lower()
                sent.append(word_to_index[w])
            except:
                sent.append(word_to_index["UNK"])
        X.append(sent)
           
    X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index["PAD"])

    if isTest:
        y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=word_to_index["PAD"])
        y = to_categorical(y, num_classes=2)
    else:
        y = None

    return (X,y)
def encoding1(X,  isTest = True):
    sentences = []
    
    for t in X:
        sentences.append(custom_tokenizer(t))

    X = []
    for s in sentences:
        sent = []
        for w in s:
            try:
                w = w.lower()
                sent.append(word_to_index[w])
            except:
                sent.append(word_to_index["UNK"])
        X.append(sent)
           
    X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index["PAD"])

  

    return X

def decoding(text_data, encoding_text, prediction):
    test = [[idx2word[i] for i in row] for row in encoding_text]

    lst_token = []

    for t in range(0, len(test)):
        yy_pred = []
        for i in range(0, len(test[t])):
            if prediction[t][i] == 1:
                yy_pred.append(test[t][i])
        lst_token.append(yy_pred)

    lis_idx = []
    for i in range(0, len(text_data)):
        idx = []
        for t in lst_token[i]:
            index = text_data[i].find(t)
            idx.append(index)
            for j in range(1, len(t)):
                index = index + 1
                idx.append(index)
        lis_idx.append(idx)

    return lis_idx

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
X1, y1 = encoding(X_train, y_train)
X2, y2 = encoding(X_dev, y_dev)
X3, y3 = encoding(X_test, y_test)


In [ ]:
x_t, y_t = encoding(X, y)


In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-ygmak00e
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-ygmak00e
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101078 sha256=68b8231189c3e75051452a5d39cd4e990e9c60c8dbed324d51d4c2522a52bbb8
  Stored in directory: /tmp/pip-ephem-wheel-cache-fp79sgbc/wheels/74/d5/f7/0245af7ac33d5b0c2e095688649916e4bf9a8d6b3362a849f5
Successfully built keras-contrib


In [ ]:
import tensorflow.keras.backend as K

def binary_loss(y_true, y_pred):
  loss = -1 * y_true * K.log( K.clip(y_pred + K.epsilon(), 0.0, 1.0) )
  loss += -1 * (1.0-y_true) * K.log( K.clip(1.0-y_pred+K.epsilon(), 0.0, 1.0) )
  return loss
     


Conditional Random Field

In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 14.6 MB/s eta 0:00:00


In [ ]:
from keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional, Flatten, Dropout
from keras.models import Model
from keras import Input
from tensorflow_addons.layers import CRF 
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
from keras.utils import plot_model
import warnings
warnings.filterwarnings("ignore")
import tensorflow_addons
# from keras.metrics import BinaryAccuracy, Precision, Recall, AUC

input = Input(shape = (max_len,))
model = Embedding(input_dim=num_words,
                    output_dim=embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=max_len,
                    trainable=True)(input)

model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units = max_len, return_sequences=True, recurrent_dropout=0.1))(model)
model = TimeDistributed(Dense(max_len, activation="relu"))(model)
model = Dense(128, activation="sigmoid")(model)
model = Dropout(0.1)(model)

crff =CRF(2)
decoded_sequence, potentials, sequence_length, chain_kernel = crff(model)
out = Dense(2, activation="sigmoid")(potentials)


model = Model(input, out)
model.compile(optimizer="adam",loss = tensorflow_addons.losses.SigmoidFocalCrossEntropy(), metrics=['accuracy'])

model.summary()

plot_model(model,to_file="bilstm-crf.pdf",show_shapes=True,show_layer_names=True)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 128, 25)           29837875  
                                                                 
 dropout_2 (Dropout)         (None, 128, 25)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 128, 256)         157696    
 nal)                                                            
                                                                 
 time_distributed_1 (TimeDis  (None, 128, 128)         32896     
 tributed)                                                       
                                                                 
 dense_6 (Dense)             (None, 128, 128)          1651

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-30-46b9172317a5>", line 35, in <cell line: 35>
    plot_model(model,to_file="bilstm-crf.pdf",show_shapes=True,show_layer_names=True)
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/vis_utils.py", line 487, in plot_model
    dot.write(to_file, format=extension)
  File "/usr/local/lib/python3.10/dist-packages/pydot_ng/__init__.py", line 1755, in write
    fobj, close = get_fobj(path, 'w+b')
  File "/usr/local/lib/python3.10/dist-packages/pydot_ng/__init__.py", line 136, in get_fobj
    fobj = open(fname, mode)
OSError: [Errno 107] Transport endpoint is not connected: 'bilstm-crf.pdf'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshel

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional, Flatten, Dropout
from keras.models import Model
from keras import Input
from tensorflow_addons.layers import CRF 
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
from keras.utils import plot_model
import warnings
warnings.filterwarnings("ignore")
import tensorflow_addons
import warnings
warnings.filterwarnings("ignore")

checkpointer = ModelCheckpoint(filepath = '"ML_Final_Proj1/data1/model/model_detection_2.h5',
                       verbose = 0,
                       mode = 'auto',
                       save_best_only = True,
                       monitor='val_loss')

model.fit(X1, np.array(y1), batch_size=64, epochs=7, validation_data=(X2, y2), callbacks=[checkpointer])


NameError: ignored

In [ ]:

from keras.models import load_model

model = load_model('"ML_Final_Proj1/data1/model/model_detection_1.h5')


In [ ]:
y_pred = model.predict(X3)
y_pred = np.argmax(y_pred, axis=-1)
y_test_true = np.argmax(y3, -1)


63/63 [==============================] - 11s 152ms/step


In [ ]:
test = [[idx2word[i] for i in row] for row in X3]
     

yy_pred = []
yy_test = []

for i in range(0, len(test[0])):
    if y_pred[0][i] == 1:
        yy_pred.append(test[0][i])

for i in range(0, len(test[0])):
    if y_test_true[0][i] == 1:
        yy_test.append(test[0][i])

print(yy_pred)
print(yy_test)


NameError: ignored

In [ ]:
raw_y = decoding(X_test, X3, y_pred)
     

f1(raw_y[0], spans_test[0])

acc = []
for i in range(0, len(spans_test)):
    acc.append(f1(raw_y[i], spans_test[i]))

print(np.mean(acc)*100)
     


57.13968131011541


In [ ]:
while(True):
  x = input()
  xx = encoding1(x)
  j=0
  xxx = []
  for i in xx:
    
    xxx[i] =idx2word[i] 
    j+=1
  print(xxx.shape)
  y_pred = model.predict(xx)
  print(y_pred)
  y_pred = np.argmax(y_pred, axis=-1)

     

  yy_pred = []
  yy_test = []

  for i in range(0, len(xxx)):
    if y_pred[0][i] == 1:
        yy_pred.append(xxx[i])
  print(yy_pred)

salam


TypeError: ignored